In [10]:
import pandas as pd
import datetime
from ast import literal_eval
import numpy as np

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import ngrams
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import scipy.sparse as sp

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score

In [2]:
# Load data
data_path=""
converters={"Body_A": literal_eval, "Body_A": literal_eval}
df = pd.read_csv('./merged_data.csv', encoding = "ISO-8859-1")
df = df[:25000]

In [3]:
df = df[:25000]
ref_time = datetime.datetime(2008, 1, 1)

def to_integer(dt_timeQ, dt_timeA):
    timeQ = dt_timeQ - ref_time
    timeA = dt_timeA - ref_time
    time_diff = 1 - (timeQ.days*86400 + timeQ.seconds) / (timeA.days*86400 + timeA.seconds)
    return time_diff

    
def get_time_diff(df):
    for index, row in df.iterrows():
        row.CreationDate_A = datetime.datetime.strptime(row.CreationDate_A, "%Y-%m-%dT%H:%M:%SZ")
        row.CreationDate_Q = datetime.datetime.strptime(row.CreationDate_Q, "%Y-%m-%dT%H:%M:%SZ")
        df.at[index, 'time_diff'] = to_integer(row.CreationDate_Q, row.CreationDate_A)
    return df


df = get_time_diff(df)
df['score'] = df['Score_A'] + df['Score_A'] * df['time_diff']

df_test = df[12500:]
df_train = df[:12500]

In [4]:
stopwords.words('english')

def tokenizer_cleaning(sentences):
    text_tokens = word_tokenize(sentences)
    tokens_without_sw = [word for word in text_tokens if not word in stopwords.words()]
    tokens = [re.sub('\W', ' ', str(row)).lower() for row in tokens_without_sw]

    return tokens

In [5]:
vectorizer_bodyQ = TfidfVectorizer(analyzer = tokenizer_cleaning, ngram_range=(5,5))
vectorizer_bodyA = TfidfVectorizer(analyzer = tokenizer_cleaning, ngram_range=(5,5))

In [6]:
vectorized_bodyQ_train = vectorizer_bodyQ.fit_transform(df_train['Body_Q'])
vectorized_bodyA_train = vectorizer_bodyA.fit_transform(df_train['Body_A'])
vectorized_train = sp.hstack([vectorized_bodyQ_train, vectorized_bodyA_train])

In [7]:
model = RandomForestRegressor(max_depth=20, random_state=0, n_estimators=1000)
model.fit(vectorized_train, df_train['score'])

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=20, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=1000, n_jobs=None, oob_score=False,
                      random_state=0, verbose=0, warm_start=False)

In [8]:
vectorized_bodyQ_test = vectorizer_bodyQ.transform(df_test['Body_Q'])
vectorized_bodyA_test = vectorizer_bodyA.transform(df_test['Body_A'])
vectorized_test = sp.hstack([vectorized_bodyQ_test, vectorized_bodyA_test])

In [45]:
predicted_score = model.predict(vectorized_test)
print(df_test['score'])
print(predicted_score)

model.score(vectorized_test, df_test['score'])

12500    -3.000015
12501    16.000916
12502     1.001173
12503     1.746214
12504    12.787825
           ...    
24995     0.000000
24996     2.000033
24997     3.000060
24998     1.000488
24999     3.002241
Name: score, Length: 12500, dtype: float64
[19.23248278 15.59432689 21.35850206 ... 13.71211521 15.30234312
 13.83693259]


-0.012658251136771215

In [ ]:
print('ok')